##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# <div dir="rtl"> الترجمة الآلية باستخدام الشبكات العصبية مع آلية الانتباه </div>

<table class="tfo-notebook-buttons" align="right" dir="rtl">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/nmt_with_attention">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    إفتح المحتوى على موقع TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/nmt_with_attention.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    تفاعل مع المحتوى على Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/nmt_with_attention.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    اعرض المصدر على Github</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/nmt_with_attention.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />نزّل الدّفتر</a>
  </td>
</table>

<div dir="rtl">

يقوم هذا الدفتر بتدريب نموذج تسلسل إلى تسلسل (seq2seq) للترجمة من الإسبانية إلى الإنجليزية. هذا مثال متقدم يفترض بعض المعرفة بنماذج التسلسل إلى تسلسل. بعد تدريب النموذج في هذا الدفتر، ستتمكن من إدخال جملة إسبانية ، مثل: 

</div> 

"*¿todavia estan en casa؟*"  


<div dir="rtl">واستخراج الترجمة الإنجليزية:

</div>  

"*are you still at home?*"

<div dir="rtl">

جودة الترجمة معقولة لمثال بسيط ، ولكن ربما يكون الرسم البياني للانتباه (attention) الناتج أكثر إثارة للاهتمام. فهو يوضح  أجزاء الجملة المدخلة التي لفتت انتباه النموذج أثناء الترجمة:

</div>

<img src="https://tensorflow.org/images/spanish-english.png" alt="spanish-english attention plot">

<div dir="rtl">

Note: 
يستغرق هذا المثال حوالي 10 دقائق للتشغيل على GPU P100.

</div> 

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

## <div dir="rtl"> تنزيل وتحضير مجموعة البيانات </div>

<div dir="rtl">

سنستخدم مجموعة بيانات لغوية مقدمة من 
[هذا الموقع](http://www.manythings.org/anki/).
 تحتوي مجموعة البيانات هذه على أزواج لغوية في الشكل  التالي:

</div>

```
May I borrow this book?	¿Puedo tomar prestado este libro?
```
<div dir="rtl">

هناك مجموعة متنوعة من اللغات المتاحة ، ولكننا سنستخدم مجموعة البيانات الإنجليزية الإسبانية. لتبسيط الأمر ، اضفنا نسخة من مجموعة البيانات هذه على Google Cloud ، ولكن يمكنك أيضًا تنزيل نسختك الخاصة. بعد تنزيل مجموعة البيانات ، إليك الخطوات التي سنتخذها لإعدادها:

1. أضف رمز *start* و *end* لكل جملة.
2. نظف الجمل بإزالة [لأحرف الخاصة](https://owasp.org/www-community/password-special-characters).
3.كون فهرس الكلمات وفهرس الكلمات العكسي (قواميس تعطي لكل كلمة معرف وحيد والعكس بالعكس).
4. كمل كل جملة إلى الطول الأقصى.


</div>

In [0]:
# Download the file
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.rstrip().strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [0]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

In [0]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [0]:
en, sp = create_dataset(path_to_file, None)
print(en[-1])
print(sp[-1])

In [0]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [0]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### <div dir="rtl"> تحديد حجم مجموعة البيانات للتجربة بشكل أسرع (اختياري)</div>

<div dir="rtl"> 

سيستغرق التدريب على مجموعة البيانات الكاملة (أكثر من 100،000 جملة) وقتًا طويلاً. للتدريب بشكل أسرع ، يمكننا تحديد حجم مجموعة البيانات بـ 30،000 جملة (بالطبع ، تنقص جودة الترجمة ببيانات أقل):

</div>

In [0]:
# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [0]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [0]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

### <div dir="rtl"> استعمال tf.data dataset </div> 

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

## <div dir="rtl"> برمجة نموذج التشفير - فك التشفير (encoder - decoder) </div>

<div dir="rtl">

لنقم ببرمجة نموذج التشفير - فك التشفير ببواسطة آلية الانتباه  (attention). يمكنك أن تقرأ عنه في الدرس التعليمي لTensorFlow [الترجمة الآلية باستخدام الشبكات العصبية (seq2seq)](https://github.com/tensorflow/nmt).
 يستخدم هذا المثال مجموعة أحدث من واجهات برمجة التطبيقات (APIs). 
و يبين هذا الدفتر [معادلات الانتباه](https://github.com/tensorflow/nmt#background-on-the-attention-mechanism) من الدرس التعليمي seq2seq.
 يوضح الرسم البياني التالي أن كل كلمة مدخلة يتم تعيين وزن لها بواسطة آلية الانتباه التي يستخدمها نموذج فك التشفير (decoder) بعد ذلك للتنبؤ بالكلمة التالية في الجملة.
 الصورة والمعدلات أدناه هما مثال لآلية الانتباه من [ورقة بحث لونغ Luong](https://arxiv.org/abs/1508.04025v5).

</div>

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

<div dir="rtl">

يتم إدخال الجمل إلى نموذج التشفير الذي يعطينا ناتج التشفير بالشكل التالي *(batch_size، max_length، hidden_size)* و الحالة الخفية (hidden state) لنموذج التشفير بالشكل التالي *(batch_size، hidden_size)*.

فيما يلي المعادلات التي يتم تطبيقها:

</div>

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

<div dir="rtl">

يستخدم هذا الدرس التعليمي [آلية انتباه Bahdanau](https://arxiv.org/pdf/1409.0473.pdf) لنموذج التشفير. دعونا نحدد الرموز قبل كتابة النسخة المبسطة:

* EO = نتاج نموذج التشفير
* H = الحالة الخفية (hidden state)
* X = المدخل إلى نموذج فك التشفير

فيمايلي الشبه شفرة:

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)`

 بدون تحديد صريح، يتم تطبيق Softmax على المحور (axis) الأخير ولكن هنا نريد تطبيقه على المحور الأول ، حيث أن شكل النتيجة هو *(batch_size، max_length، hidden_size)*. "max_length" هو طول مدخلاتنا. نظرًا لأننا نحاول تعيين وزن لكل مدخل ، يجب تطبيق Softmax على هذا المحور.
* `context vector = sum(attention weights * EO, axis = 1)`. نفس السبب أعلاه 
لاختيار المحور  1. 
* `embedding output` = يتم تمرير المدخل X إلى نموذج فك التشفير  من خلال طبقة التضمين (embedding).
* `merged vector = concat(embedding output, context vector)`
* ثم يتم إدخال هذا المتجه (vector) المدمج لـ GRU

تم تحديد أشكال جميع المتجهات (vectors) لكل خطوة في التعليقات في الكود:

</div>

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [0]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

## <div dir='rtl'> تحديد خوارزميّة تحسين (optimizer) ودالّة خسارة (loss function) </div>

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## <div dir='rtl'> حفظ نقاط الفحص   عبر الكائنات (objects) </div>

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## <div dir='rtl'> التدريب </div>

<div dir='rtl'>

1. مرر *المدخل* عبر *نموذج التشفير* الذي يخرج نتاج نموذج التشفير  و *الحالة المخفية*.
2. يتم تمرير نتاج نموذج التشفير ، والحالة المخفية والمدخل لنموذج فك التشفير (وهو الرمز المميز *start*) إلى نموذج فك التشفير.
3. يقوم نموذج فك التشفير بإرجاع *التنبؤات* والحالة المخفية *لنموذج فك التشفير*.
4. يتم بعد ذلك تمرير الحالة المخفية لنموذج فك التشفير إلى النموذج واستخدام التنبؤات لحساب دالة الخسارة.
5. استخدم *إجبار المعلم* لتحديد المدخل التالي إلى نموذج فك التشفير.
6. *إجبار المعلم* هو التقنية الذي يتم خلالها تمرير *الكلمة الهدف أي الصحيحة* كالمدخل التالي إلى نموذج فك التشفير.
7. الخطوة الأخيرة هي حساب الميول (gradients) وتطبيقها في خوارزميّة التحسين ثم تطبيق الانتشار العكسي (backpropagation).

</div>

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss واختبار/ steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

## <div dir='rtl'> الترجمة <div>

<div dir='rtl'>

* تشبه وظيفة التقييم حلقة التدريب ، باستثناء أننا لا نستخدم تقنية *إجبار المعلم* هنا. المدخل في نموذج فك التشفير في كل خطوة زمنية هو تنبؤاته السابقة بالإضافة إلى الحالة المخفية و نتاج نموذج التشفير.
* توقف عن التنبؤ عندما يتنبأ النموذج برمز *end*.
* احفظ *أوزان الانتباه* لكل خطوة زمنية.

Note: يتم احتساب نتاج نموذج التشفير مرة واحدة فقط لمدخل واحد.

<div>



In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

## <div dir='rtl'>استعادة أحدث نقطة فحص واختبارها</div>

In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
translate(u'hace mucho frio aqui.')

In [0]:
translate(u'esta es mi vida.')

In [0]:
translate(u'¿todavia estan en casa?')

In [0]:
# wrong translation
translate(u'trata de averiguarlo.')

## <div dir='rtl'> الخطوات التالية </div>

<div dir='rtl'>

* [قم بتنزيل مجموعة بيانات مختلفة](http://www.manythings.org/anki/) لتجربة ترجمات أخرى، على سبيل المثال ، الإنجليزية إلى الألمانية ، أو الإنجليزية إلى الفرنسية.
* جرب التدريب على مجموعة بيانات أكبر ، أو استخدام المزيد من الحقبات (epochs)

</div>
